<a href="https://colab.research.google.com/github/UrviSoni/secure_and_private_AI/blob/master/Implementing_local_differencial_privacy_L_5_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import torch

In [0]:
def get_parallel_db(db, remove_index):
  
  return torch.cat((db[0:remove_index],db[remove_index + 1:]))

In [0]:
def get_parallel_dbs(db):
  
  parallel_dbs = list()
  
  for i in range(len(db)):
    pdb = get_parallel_db(db, i)
    parallel_dbs.append(pdb)
    
  return parallel_dbs

In [0]:
def create_db_and_parallels(num_entries):
  
  db = torch.rand(num_entries) > 0.5
  pdbs = get_parallel_dbs(db)
  
  return db, pdbs

In [0]:
db, pdbs = create_db_and_parallels(20)

In [6]:
db

tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
       dtype=torch.uint8)

In [7]:
pdbs

[tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1],
        dtype=torch.uint8),
 tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0,

In [0]:
def query(db):
  
  return db.sum()

In [9]:
query(db)

tensor(9)

In [10]:
query(pdbs[3])

tensor(9)

In [0]:
full_db_result = query(db)

In [0]:
max_distance = 0
for pdb in pdbs:
  pdb_result = query(pdb)
  
  db_distance = torch.abs(pdb_result - full_db_result)
  
  if(db_distance > max_distance):
    max_distance = db_distance

In [0]:
def sensitivity(query, n_entries=1000):
  
  db, pdbs = create_db_and_parallels(n_entries)
  full_db_result = query(db)
  
  max_distance = 0
  for pdb in pdbs:
    pdb_result = query(pdb)

    db_distance = torch.abs(pdb_result - full_db_result)

    if(db_distance > max_distance):
      max_distance = db_distance
      
  return max_distance

In [14]:
sensitivity(query)

tensor(1)

In [0]:
def query(db):
  
  return db.float().mean()

In [16]:
sensitivity(query)

tensor(0.0005)

In [0]:
def query(db, threshold = 5):
  
  return (db.sum() > threshold).float()

In [18]:
db, pdbs = create_db_and_parallels(10)
db.sum()

tensor(5)

In [19]:
db, pdbs = create_db_and_parallels(10)
query(db)

tensor(1.)

In [20]:
db.sum() > 5

tensor(1, dtype=torch.uint8)

In [0]:
sens_f = sensitivity(query, n_entries = 5)

In [22]:
for i in range(10):
  
  sens_f = sensitivity(query, n_entries = 10)
  print(sens_f)

0
0
0
0
0
0
0
0
0
0


In [0]:
db, _ = create_db_and_parallels(100)

In [0]:
pdb = get_parallel_db(db, remove_index = 10)

In [25]:
db[10] 

tensor(1, dtype=torch.uint8)

In [26]:
sum(db)

tensor(49, dtype=torch.uint8)

In [27]:
# differencing attack using sum query

sum(db) - sum(pdb)

tensor(1, dtype=torch.uint8)

In [28]:
# differencing attack using mean query

(sum(db).float() / len(db)) - (sum(pdb).float() / len(pdb))

tensor(0.0052)

In [29]:
# differencing attack threshold

(sum(db).float() > 52) - (sum(pdb).float() > 52)

tensor(0, dtype=torch.uint8)

In [0]:
db, pdbs = create_db_and_parallels(100)


In [31]:
# actual values from people

db

tensor([0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
        0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0,
        1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0,
        1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0,
        1, 1, 0, 0], dtype=torch.uint8)

In [0]:
first_coin_flip = (torch.rand(len(db)) > 0.5).float()
second_coin_flip = (torch.rand(len(db)) > 0.5).float()

In [33]:
first_coin_flip

tensor([1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0.,
        0., 1., 1., 0., 0., 0., 1., 1., 0., 0., 1., 1., 1., 1., 0., 0., 1., 1.,
        0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 1., 0., 1., 0., 0.,
        1., 1., 1., 0., 1., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 1., 1., 1., 1., 0., 0.])

In [34]:
second_coin_flip

tensor([0., 0., 1., 1., 0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1., 1., 0., 0., 0., 0.,
        0., 1., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 0., 1., 1., 0., 0., 1.,
        0., 1., 1., 0., 1., 0., 1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0.,
        1., 1., 1., 0., 1., 0., 1., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 1., 1., 1., 0.])

In [35]:
db.float() * first_coin_flip

tensor([0., 1., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 0., 1., 0., 1., 0., 0.,
        1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 0., 1., 0., 0., 0., 1., 1., 0., 0.])

In [36]:
(1 - first_coin_flip)

tensor([0., 0., 1., 0., 1., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1.,
        1., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0., 1., 1., 0., 0.,
        1., 1., 0., 1., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 0., 1., 1., 1.,
        1., 1., 1., 1., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 1., 1.,
        0., 0., 0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 1., 1., 1., 1., 1., 0.,
        1., 1., 0., 1., 0., 0., 0., 0., 1., 1.])

In [37]:
(1 - first_coin_flip) * second_coin_flip

tensor([0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
        1., 0., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 1., 0., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
        0., 1., 1., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0.,
        0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0., 1., 1., 0., 1., 0., 0.,
        0., 1., 0., 1., 0., 0., 0., 0., 1., 0.])

In [0]:
augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

In [0]:
db_result = torch.mean(augmented_database.float()) * 2 - 0.5

In [40]:
true_result = torch.mean(db.float())
true_result

tensor(0.4500)

In [0]:
def query(db):
  
  true_result = torch.mean(db.float())

  first_coin_flip = (torch.rand(len(db)) > 0.5).float()
  second_coin_flip = (torch.rand(len(db)) > 0.5).float()
  
  augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
  db_result = torch.mean(augmented_database.float()) * 2 - 0.5
  
  return db_result, true_result

In [42]:
db, pdbs = create_db_and_parallels(10)
private_result, true_result = query(db)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.3000)
Without Noisetensor(0.4000)


In [43]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.3800)
Without Noisetensor(0.4200)


In [44]:
db, pdbs = create_db_and_parallels(1000)
private_result, true_result = query(db)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.5320)
Without Noisetensor(0.4990)


In [45]:
db, pdbs = create_db_and_parallels(10000)
private_result, true_result = query(db)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.5146)
Without Noisetensor(0.5036)


In [0]:
def query(db, noise=0.2):
  
  true_result = torch.mean(db.float())

  first_coin_flip = (torch.rand(len(db)) > 0.5).float()
  second_coin_flip = (torch.rand(len(db)) > 0.5).float()
  
  augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip
  db_result = torch.mean(augmented_database.float()) * 2 - 0.5
  
  return db_result, true_result

In [0]:
true_dist_mean = 0.7 # 70% people said YES
noise_dist_mean = 0.5 # 50/50 coin flip

In [48]:
augmented_database_mean = (true_dist_mean + noise_dist_mean) / 2
augmented_database_mean

0.6

In [0]:
noise = 0.2

true_result = torch.mean(db.float())

first_coin_flip = (torch.rand(len(db)) > 0.5).float()
second_coin_flip = (torch.rand(len(db)) > 0.5).float()
  
augmented_database = db.float() * first_coin_flip + (1 - first_coin_flip) * second_coin_flip

sk_result = augmented_database.float().mean()

In [50]:
sk_result

tensor(0.5028)

In [0]:
noise = 0.5 # 50/50 first coin flip

true_dist_mean = 0.7 # 70% people said YES
noise_dist_mean = 0.5 # 50/50 coin flip

In [52]:
augmented_database_mean = (true_dist_mean * noise) + (noise_dist_mean * (1 - noise)) 
augmented_database_mean

0.6

In [0]:
private_result = ((sk_result / noise) - 0.5) * noise / (1 - noise)

In [54]:
sk_result

tensor(0.5028)

In [55]:
private_result

tensor(0.5056)

In [56]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.1)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.4400)
Without Noisetensor(0.4600)


In [57]:
db, pdbs = create_db_and_parallels(100)
private_result, true_result = query(db, noise=0.8)
print("With Noise"  + str(private_result))
print("Without Noise" + str(true_result))

With Noisetensor(0.5600)
Without Noisetensor(0.5100)


In [58]:
db, pdbs = create_db_and_parallels(100)

def query(db):
  
  return torch.sum(db.float() + noise)

query(db)

tensor(108.)

In [59]:
db, pdbs = create_db_and_parallels(100)

def query(db):
  
  return torch.sum(db.float()) 

def M(db):
  
  query(db) + noise

query(db)

tensor(47.5000)

In [0]:
epsilon = 0.5

In [0]:
import numpy as np

In [0]:
db, pdbs = create_db_and_parallels(100)

In [63]:
db

tensor([0, 0, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
        0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1,
        1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1,
        0, 1, 0, 1], dtype=torch.uint8)

In [64]:
sum(db)

tensor(48, dtype=torch.uint8)

In [65]:
(db * 2)

tensor([0, 0, 0, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 2, 2, 0, 2, 2, 0, 0, 0, 2, 2, 2,
        2, 2, 2, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0,
        0, 2, 2, 0, 2, 2, 2, 2, 0, 2, 0, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 0, 0, 2,
        2, 2, 2, 2, 0, 2, 2, 0, 0, 2, 0, 0, 0, 2, 0, 2, 2, 2, 0, 0, 0, 2, 0, 2,
        0, 2, 0, 2], dtype=torch.uint8)

In [66]:
sum(db * 2)

tensor(96, dtype=torch.uint8)

In [0]:
def sum_query(db):
  return db.sum()

In [0]:
def laplacian_mechanism(db, query, sensitivity):
  
  beta = sensitivity / epsilon
  
  noise = torch.tensor(np.random.laplace(0, beta, 1))
  
  return query(db) + noise

In [0]:
def mean_query(db):
  
  return torch.mean(db.float())

In [74]:
mean_query(db)

tensor(0.4800)

In [71]:
laplacian_mechanism(db, sum_query, 1)

tensor([48.0762], dtype=torch.float64)

In [85]:
laplacian_mechanism(db, sum_query, 1/100)

tensor([48.0245], dtype=torch.float64)